In [26]:
import pandas as pd
import math

Unemployment Rate

In [4]:
df = pd.read_csv('/Users/shashankkumar/Documents/GitHub/MacroEcon/simulator_data/NYC/Unemployment_unprocessed/unemployment_rate_csvs/Brooklyn-Table.csv')
df = df.rename(columns={
    "Revised 2019-2023 Labor Force Data": "area",
    "Unnamed: 1": "year",
    "Unnamed: 2": "month",
    "Unnamed: 3": "labor_force",
    "Unnamed: 4": "employed",
    "Unnamed: 5": "unemployed",
    "Unnamed: 6": "unemployment_rate"
})
df = df.drop(df.index[:2])
df = df.dropna()
df.reset_index(drop=True, inplace=True)
df = df[['area','year', 'month',"labor_force", 'unemployment_rate']]
df['unemployment_rate'] = df['unemployment_rate'].str.replace('\t', '')
df['unemployment_rate'] = df['unemployment_rate'].astype(float)
df['labor_force'] = df['labor_force'].str.replace(',', '')
df['labor_force'] = df['labor_force'].astype(int)
month_to_index = {
    'Jan': 0, 'Feb': 1, 'Mar': 2, 'Apr': 3, 'May': 4, 'Jun': 5,
    'Jul': 6, 'Aug': 7, 'Sep': 8, 'Oct': 9, 'Nov': 10, 'Dec': 11,'Avg': 12
}
df['month'] = df['month'].map(month_to_index)
df['year'] = df['year'].astype(int) - df['year'].astype(int).min()
df['area'] = df['area'].replace('Kings County', 1)
df = df[df['year'] > 0]
df.to_csv('/Users/shashankkumar/Documents/GitHub/MacroEcon/simulator_data/NYC/brooklyn_unemp.csv', index=False)
df

,area,year,month,labor_force,unemployment_rate
0,1,4,0,1217200,5.7
1,1,4,1,1230500,5.8
2,1,4,2,1232900,5.4
3,1,4,3,1219600,5.0
4,1,4,4,1223800,5.2
...,...,...,...,...,...
60,1,0,8,1252500,3.7
61,1,0,9,1260400,4.0
62,1,0,10,1250900,3.7
63,1,0,11,1250100,3.4


#### UI claims

In [6]:
df = pd.read_csv("/Users/shashankkumar/Documents/GitHub/MacroEcon/simulator_data/NYC/Unemployment_unprocessed/unemployment_claims/UI_CLAIMS.csv")

In [15]:
df

,area,county,Beneficiaries.0,Total Amount.0,Beneficiaries.1,Total Amount.1,Beneficiaries.2,Total Amount.2,Beneficiaries.3,Total Amount.3,...,Beneficiaries.57,Total Amount.57,Beneficiaries.58,Total Amount.58,Beneficiaries.59,Total Amount.59,Beneficiaries.60,Total Amount.60,Beneficiaries.61,Total Amount.61
0,NEW YORK CITY,BRONX,"9,600","$ 1,43,14,000","9,400","$ 1,35,60,000","8,800","$ 1,30,10,000","8,200","$ 1,44,99,000",...,"13,500","$ 2,14,72,000","13,100","$ 1,78,98,000","13,100","$ 1,78,17,000","16,100","$ 2,29,73,000","15,200","$ 1,95,33,000"
1,NEW YORK CITY,KINGS,"14,700","$ 2,27,90,000","14,700","$ 2,10,95,000","13,600","$ 2,04,18,000","12,600","$ 2,28,96,000",...,"24,700","$ 4,19,97,000","23,600","$ 3,49,87,000","22,500","$ 3,27,71,000","26,600","$ 4,16,63,000","24,800","$ 3,48,54,000"
2,NEW YORK CITY,NEW YORK,"9,400","$ 1,52,37,000","9,200","$ 1,35,06,000","8,500","$ 1,32,46,000","7,900","$ 1,51,42,000",...,"15,900","$ 2,78,32,000","15,100","$ 2,34,39,000","14,500","$ 2,17,31,000","16,100","$ 2,68,66,000","14,900","$ 2,21,83,000"
3,NEW YORK CITY,QUEENS,"11,100","$ 1,81,67,000","10,900","$ 1,65,54,000","10,100","$ 1,59,70,000","9,100","$ 1,77,37,000",...,"17,000","$ 2,85,39,000","16,500","$ 2,44,98,000","16,100","$ 2,32,99,000","19,500","$ 3,05,44,000","18,500","$ 2,59,55,000"
4,NEW YORK CITY,RICHMOND,"2,300","$ 37,93,000","2,200","$ 35,45,000","2,000","$ 32,36,000","1,800","$ 35,94,000",...,"3,100","$ 49,30,000","3,000","$ 41,88,000","3,100","$ 40,67,000","4,200","$ 56,68,000","3,700","$ 46,32,000"


In [17]:
df = df.rename(columns={
    "Region": "area",
    "County": "county",
    "Beneficiaries": "Beneficiaries.0",
    "Total Amount": "Total Amount.0"
})

# Initialize an empty DataFrame to store the reshaped data
reshaped_df = pd.DataFrame()

# Iterate over each pair of "Beneficiaries" and "Total Amount" columns
for i in range(62):
    temp_df = df[['area', 'county', f'Beneficiaries.{i}', f'Total Amount.{i}']].copy()
    temp_df['month'] = i
    temp_df = temp_df.rename(columns={
        f'Beneficiaries.{i}': 'beneficiaries',
        f'Total Amount.{i}': 'total_amount'
    })
    temp_df['beneficiaries'] = temp_df['beneficiaries'].str.replace(',', '').astype(int)
    temp_df['total_amount'] = temp_df['total_amount'].str.replace('$', '').str.replace(',', '').astype(float)
    reshaped_df = pd.concat([reshaped_df, temp_df])

# Reset the index of the reshaped DataFrame
reshaped_df.reset_index(drop=True, inplace=True)
reshaped_df.drop(columns=['area'], inplace=True)
reshaped_df.to_csv('/Users/shashankkumar/Documents/GitHub/MacroEcon/simulator_data/NYC/processed_UI_claims.csv', index=False)
# Save the reshaped DataFrame to a CSV file
# reshaped_df.to_csv('processed_UI_claims.csv', index=False)

In [19]:
reshaped_df = reshaped_df[reshaped_df['county'] == 'KINGS']
reshaped_df

,county,beneficiaries,total_amount,month
1,KINGS,14700,22790000.0,0
6,KINGS,14700,21095000.0,1
11,KINGS,13600,20418000.0,2
16,KINGS,12600,22896000.0,3
21,KINGS,11300,19486000.0,4
...,...,...,...,...
286,KINGS,24700,41997000.0,57
291,KINGS,23600,34987000.0,58
296,KINGS,22500,32771000.0,59
301,KINGS,26600,41663000.0,60


In [30]:
reshaped_df['year'] = reshaped_df['month'].apply(lambda x: math.floor((x)/12) )

In [35]:
reshaped_df = reshaped_df['beneficiaries']
reshaped_df

1      14700
6      14700
11     13600
16     12600
21     11300
       ...  
286    24700
291    23600
296    22500
301    26600
306    24800
Name: beneficiaries, Length: 62, dtype: int64

In [36]:
reshaped_df.to_pickle('/Users/shashankkumar/Documents/GitHub/MacroEcon/simulator_data/NYC/brooklyn_processed_UI_claims.pkl')